# NXTomoReader Demo

Note: this demo is a work in progress and was created for the purpose of demonstrating a contibution for the reader hackathon!

The NXTomoReader in its current state allows:

- Reading of geometry
- Reading of projections 
- Reading of dark fields
- Reading of flat fields

## Data format: NXTomo

The data is in the format NXTomo - https://manual.nexusformat.org/classes/applications/NXtomo.html

## CIL Version

This notebook was developed using CIL v25.0.0

## Dataset
The data is available from: https://github.com/DiamondLightSource/Savu/blob/master/test_data/data/24737_fd.nxs
We acknowledge Diamond Light Source for time on Beamline I13-2 under Proposal mt9396-1

Update this filepath to where you have saved the dataset:

In [ ]:
file_path = 'data/24737_fd.nxs'

## Loading Geometry

In [ ]:
from readers.NXTomoReader import NXTomoReader

from cil.utilities.display import show2D, show_geometry

In [ ]:
reader = NXTomoReader(file_path)

In [ ]:
acq_geom = reader.get_geometry()

show_geometry(acq_geom);

In [ ]:
print(acq_geom)

## Loading Projections, Flat and Dark Fields

In [ ]:
acq_data = reader.read()

show2D(acq_data, origin='upper-left');

In [ ]:
dark_field = reader.load_dark_field()

show2D(dark_field, origin='upper-left');

In [ ]:
flat_field = reader.load_flat_field()
show2D(flat_field, origin='upper-left');

In [ ]:
if False: # load subsets
    dims = (slice(0,91), slice(10,120,4), slice(0,160,4))
    dims2 = (slice(0, -1), dims[1], dims[2])
    acq_data = reader.read(dimensions=dims)
    dark_field = reader.load_dark_field()[dims2]
    flat_field = reader.load_flat_field()[dims2]

# Pre-processing and Reconstruction

For details of the pre-processing steps performed, see our [CIL-Demos example which uses the same dataset](https://github.com/TomographicImaging/CIL-Demos/blob/main/demos/1_Introduction/03_preprocessing.ipynb)! However, in the CIL-demo the data had been converted into the CIL Nexus format first.

In [ ]:
from cil.processors import Normaliser, TransmissionAbsorptionConverter, Slicer, CentreOfRotationCorrector
from cil.recon import FBP

data_norm = Normaliser(flat_field=flat_field.mean(axis=0), dark_field=dark_field.mean(axis=0))(acq_data)

data_exp = TransmissionAbsorptionConverter()(data_norm)

data_crop = Slicer(roi={'vertical': (1, None)})(data_exp)

data_cor = CentreOfRotationCorrector.xcorrelation()(data_crop)

recon = FBP(data_cor).run()

In [ ]:
show2D(recon,slice_list=('horizontal_y', recon.shape[1]//2), title='Reconstruction FBP',  origin='upper-left');

## Outlook

Some limitations of this reader is that currently it does not have a method for loading an ROI and some of the method names are not ideal.

It also has a lot of methods that we probably don't need!

Perhaps we can work on these things during the hackathon!